In [1]:
import os
import glob
import subprocess
from osgeo import gdal
from datetime import datetime
from collections import Counter
import numpy as np

from platform import python_version
print(python_version())

# Dumb, update as needed based on conda env and user paths
# Set paths to PROJ & GDAL data
os.environ["PROJ_LIB"] = "/home/mmacander/miniconda3/envs/cog/share/proj"
os.environ["GDAL_DATA"] = "/home/mmacander/miniconda3/envs/cog/share/gdal"

dem_tif = "/data/gis/gis_base/DEM/ifsar/wgs1984_ellipsoid_height/alaska_ifsar_dsm_20200925_plus_us_noaa_g2009.tif"  # Digital Elevation Model


3.9.21


In [10]:
download_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/downloads/nome_beaver"
download_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped"
os.chdir(download_dir)

# dirs = glob.glob("05*/", root_dir = download_dir)
order_dirs = glob.glob(download_dir + "/05*/")
# order_dirs = glob.glob(download_dir + "/050295*/")
print(order_dirs)

output_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/nome_beaver/ortho_ms4_2m"
output_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_ms4_2m"


['/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296503010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296499010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296466010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296478010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296498010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296483010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296495010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296467010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296473010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296492010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296479010_01/', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296481010_01/', '/data/gis/rast

In [11]:
order_dir = order_dirs[0]
print(order_dir)

/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296503010_01/


In [12]:
# Define search pattern to find all .ntf files in subdirectories
ntf_files = glob.glob(order_dir + "*_MUL/*.NTF", recursive=True)
# ntf_files = glob.glob("**/*.ntf", recursive=True)
# output_cog = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/nome_beaver/outputs_python/final_mosaic_cog4.tif "  # Output COG filename

# Ensure we found NTF files
if not ntf_files:
    raise ValueError("❌ No .ntf files found in subdirectories!")

print(f"✅ Found {len(ntf_files)} NTF files to process.")
print(ntf_files)

ntf_file = ntf_files[0]
print("\n", ntf_file)

✅ Found 5 NTF files to process.
['/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296503010_01/050296503010_01_P001_MUL/13JUN20201558-M1BS-050296503010_01_P001.NTF', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296503010_01/050296503010_01_P004_MUL/13JUN20201607-M1BS-050296503010_01_P004.NTF', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296503010_01/050296503010_01_P003_MUL/13JUN20201604-M1BS-050296503010_01_P003.NTF', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296503010_01/050296503010_01_P002_MUL/13JUN20201601-M1BS-050296503010_01_P002.NTF', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296503010_01/050296503010_01_P005_MUL/13JUN20201610-M1BS-050296503010_01_P005.NTF']

 /data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296503010_01/050296503010_01_P001_MUL/13JUN20201558-M1BS-050296503010_01_P001.NTF


In [13]:
# Examine one file

# Open the NTF file
dataset = gdal.Open(ntf_file)
if not dataset:
    raise ValueError(f"❌ Could not open {ntf_file}")

# Get general metadata
metadata = dataset.GetMetadata()
print("✅ NTF Metadata:")
for key, value in metadata.items():
    print(f"{key}: {value}")

# Get the number of bands
num_bands = dataset.RasterCount
print(f"📌 Number of bands: {num_bands}")

# Get geotransform (spatial metadata)
geotransform = dataset.GetGeoTransform()
print(f"📌 Geotransform: {geotransform}")

# Get projection (Coordinate Reference System)
projection = dataset.GetProjection()
print(f"📌 Projection: {projection}")

# Get dimensions
width = dataset.RasterXSize
height = dataset.RasterYSize
print(f"📌 Dimensions: {width} x {height} pixels")

# Get information about each band
for i in range(1, num_bands + 1):
    band = dataset.GetRasterBand(i)
    print(f"📌 Band {i}: Type={gdal.GetDataTypeName(band.DataType)}, NoDataValue={band.GetNoDataValue()}")

# Close the dataset
dataset = None

NITF_ISORCE = metadata['NITF_ISORCE']
print(NITF_ISORCE)


✅ NTF Metadata:
NITF_ABPP: 11
NITF_CCS_COLUMN: 0
NITF_CCS_ROW: 0
NITF_CLEVEL: 05
NITF_ENCRYP: 0
NITF_FBKGC: 126,126,126
NITF_FDT: 20250221091317
NITF_FHDR: NITF02.10
NITF_FSCATP: 
NITF_FSCAUT: 
NITF_FSCLAS: U
NITF_FSCLSY: US
NITF_FSCLTX: 
NITF_FSCODE: 
NITF_FSCOP: 00000
NITF_FSCPYS: 00000
NITF_FSCRSN: 
NITF_FSCTLH: 
NITF_FSCTLN: 
NITF_FSDCDT: 
NITF_FSDCTP: 
NITF_FSDCXM: 
NITF_FSDG: 
NITF_FSDGDT: 
NITF_FSREL: 
NITF_FSSRDT: 
NITF_FTITLE: 13JUN20201558-M1BS-050296503010_01_P001.NTF
NITF_IALVL: 0
NITF_IC: NC
NITF_ICAT: MS
NITF_ICORDS: G
NITF_IDATIM: 20130620201558
NITF_IDLVL: 1
NITF_IGEOLO: 632825N1441106W632836N1435030W632016N1435030W632008N1441048W
NITF_IID1: M11194EA00
NITF_IID2: 20JUN13QB021400013JUN20201558-M1BS-050296503010_01_P001
NITF_ILOC_COLUMN: 0
NITF_ILOC_ROW: 0
NITF_IMAG: 1.0 
NITF_IMAGE_COMMENTS: The imagery and metadata data on this media is the property of                  Maxar Technologies and is licensed for use only.                                All use must be in acc

In [14]:
# Test extraction of catalog id
# Ensure we have NTF files
if not ntf_files:
    raise ValueError("❌ No .ntf files found!")

print(f"✅ Found {len(ntf_files)} NTF files.")

# Loop through each NTF file to extract the `NITF_PIAIMC_SOURCE` metadata
for ntf_file in ntf_files:
    dataset = gdal.Open(ntf_file)
    if dataset:
        metadata = dataset.GetMetadata()
        source_metadata = metadata.get("NITF_PIAIMC_SOURCE", "Unknown")
        
        # Check if the source contains "VNIR"
        # if "VNIR" in source_info:
        #     print(f"📌 {ntf_file}: {source_info}")
        # vnir_str = source_info.get("VNIR", "VNIR key not found")
    # Check if VNIR exists inside the metadata string
    if "VNIR:" in source_metadata:
        for line in source_metadata.split("\n"):  # Handle multi-line metadata
            if line.startswith("VNIR:"):
                vnir_value = line.split(":", 1)[1].strip()  # Get value after colon
       
        print(vnir_value)
        # return(vnir_value)
        dataset = None  # Close dataset



✅ Found 5 NTF files.
101001001194EA00
101001001194EA00
101001001194EA00
101001001194EA00
101001001194EA00


In [ ]:
# Run on batch of unzipped order_dirs
for order_dir in order_dirs:
    print(f"\nOrder dir: {order_dir}")
    # Define search pattern to find all .ntf files in subdirectories
    ntf_files = glob.glob(order_dir + "*_MUL/*.NTF", recursive=True)
    
    # Ensure we have files
    if not ntf_files:
        # raise ValueError("❌ No .ntf files found!")
        print("❌ No .ntf files found!")
        continue
    
    print(f"✅ Found {len(ntf_files)} NTF files.")
    
    # Store metadata values
    metadata_values = {}
    
    # Extract metadata from each file
    for ntf_file in ntf_files:
        dataset = gdal.Open(ntf_file)
        if dataset:
            metadata = dataset.GetMetadata()
            for key, value in metadata.items():
                if value and value.strip():  # Remove empty or whitespace-only values
                    if key not in metadata_values:
                        metadata_values[key] = []
                    metadata_values[key].append(value.strip())
            dataset = None  # Close file
    
    # Function to aggregate metadata
    aggregated_metadata = {}
    for key, values in metadata_values.items():
        try:
            # If the key ends with "_DATE", use the minimum date
            if key.upper().endswith("_DATE"):
                aggregated_metadata[key] = min(values)
            else:
                # Convert values to float if possible, then take the median
                numeric_values = [float(v) for v in values if v.replace('.', '', 1).isdigit()]
                if numeric_values:
                    aggregated_metadata[key] = str(np.median(numeric_values))
                else:
                    # Take the lexicographical minimum for non-numeric values
                    aggregated_metadata[key] = min(values)
        except ValueError:
            # Take minimum for categorical values
            aggregated_metadata[key] = min(values)
    
    # print("✅ Aggregated Metadata:")
    # for k, v in aggregated_metadata.items():
    #     print(f"{k}: {v}")
    
    # source_metadata = aggregated_metadata.get("NITF_PIAIMC_SOURCE", "Unknown")
    source_metadata = aggregated_metadata.get("NITF_PIAIMC_SOURCE", "Unknown")
        
        # Check if the source contains "VNIR"
        # if "VNIR" in source_info:
        #     print(f"📌 {ntf_file}: {source_info}")
        # vnir_str = source_info.get("VNIR", "VNIR key not found")
    # Check if VNIR exists inside the metadata string
    if "VNIR:" in source_metadata:
        for line in source_metadata.split("\n"):  # Handle multi-line metadata
            if line.startswith("VNIR:"):
                catid = line.split(":", 1)[1].strip()  # Get value after colon
       
        print(catid)
    
    sensor = aggregated_metadata.get("NITF_ISORCE")
    
    # Find the minimum date
    # if acquisition_dates:
    #     min_date = min(acquisition_dates)  # Dates are stored as strings, lexicographically sorted works for YYYYMMDD
    #     max_date = max(acquisition_dates)  # Dates are stored as strings, lexicographically sorted works for YYYYMMDD
    #     print(f"✅ Earliest acquisition date: {min_date}")
    #     print(f"✅   Latest acquisition date: {max_date}")
    # else:
    #     print("❌ No acquisition dates found in metadata!")
    
    min_acquisition_date = aggregated_metadata.get("NITF_STDIDC_ACQUISITION_DATE")
    min_acquisition_date = f"{min_acquisition_date[:8]}_{min_acquisition_date[8:]}"
    print(f"✅ Earliest acquisition date: {min_acquisition_date}")
    
    # # If only one unique VNIR value exists, print it
    # if len(vnir_counter) == 1:
    #     catid = next(iter(vnir_counter))  # Get the single VNIR value
    #     print(f"Catalog ID: {catid}")
    # else:
    #     print("Multiple catalog ids, exiting.")
    #     sys.exit()  # Exit without printing anything
    print(f"Catalog ID: {vnir_value}")
    
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    
    output_name = f"MS4_2m_{min_acquisition_date}_{sensor}_{catid}_v{timestamp}.tif"
    
    print(f"Output Filename: {output_name}")
    
    # Function to determine the number of bands in an NTF file
    def get_num_bands(ntf_file):
        dataset = gdal.Open(ntf_file)
        if dataset:
            num_bands = dataset.RasterCount
            dataset = None  # Close the dataset
            return num_bands
        else:
            raise ValueError(f"❌ Could not open {ntf_file}")
    
    # Check the band count of the first NTF file (assumes all have the same structure)
    num_bands = get_num_bands(ntf_files[0])
    print(f"✅ Detected {num_bands} bands.")
    
    # Define which bands to process
    bands_to_process = list(range(1, 5)) if num_bands == 4 else [2, 3, 5, 7]
    
    # Apply aggregated metadata to output COG
    output_cog = os.path.join(output_dir, output_name)  # Output COG filename
    
    # gdal.SetConfigOption("GDAL_TIFF_INTERNAL_MASK", "YES")  # Enable internal mask support
    gdal.SetConfigOption("GDAL_CACHEMAX", "1536")
    
    warp_options = gdal.WarpOptions(
        dstSRS="EPSG:3338",  # Set target spatial reference
        srcBands=bands_to_process,
        multithread=True,
        warpOptions=["NUM_THREADS=20"],
        creationOptions=["NUM_THREADS=20",
                         "PREDICTOR=2",  # Apply LZW compression predictor
                         "BIGTIFF=IF_SAFER"],  # Enable BigTIFF support
        # "--config", "GDAL_CACHEMAX 1536",
        warpMemoryLimit=1536,
        rpc=True,  # Use Rational Polynomial Coefficients
        transformerOptions=[f"RPC_DEM={dem_tif}"],  # Use the specified DEM
        srcNodata=0,
        format="COG",  # Output format is COG
        xRes=2,
        yRes=2,
        targetAlignedPixels=True,
        resampleAlg="cubic",  # Resampling method
        # metadataOptions=aggregated_metadata  # Apply aggregated metadata
        copyMetadata=True)
    
    # Run gdal.Warp with aggregated metadata
    gdal.Warp(output_cog, ntf_files, options=warp_options)

    # # Construct the gdalinfo command
    info_cmd = [
        conda_path, "run", "-n", conda_env,
        # "gdalwarp",
        "/home/mmacander/miniconda3/envs/cog/bin/gdalinfo",
        "-stats",
        "-hist"
    ]
    
    info_cmd.extend([output_cog])
    
    # # Print and execute the command
    print(f"🚀 Running: {' '.join(info_cmd)}")
    # subprocess.run(info_cmd, shell=True, executable="/bin/bash", check=True)
    subprocess.run(info_cmd, check=True)
    
    print(f"✅ Orthorectified COG saved with metadata, stats: {output_cog}")



Order dir: /data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/unzipped/050296503010_01/
✅ Found 5 NTF files.
101001001194EA00
✅ Earliest acquisition date: 20130620_201558
Catalog ID: 101001001194EA00
Output Filename: MS4_2m_20130620_201558_QB02_101001001194EA00_v20250226164946.tif
✅ Detected 4 bands.
🚀 Running: /home/mmacander/miniconda3/bin/conda run -n cog /home/mmacander/miniconda3/envs/cog/bin/gdalinfo -stats -hist /data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_ms4_2m/MS4_2m_20130620_201558_QB02_101001001194EA00_v20250226164946.tif
Driver: GTiff/GeoTIFF
Files: /data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_ms4_2m/MS4_2m_20130620_201558_QB02_101001001194EA00_v20250226164946.tif
Size is 12393, 28480
Coordinate System is:
PROJCRS["NAD83 / Alaska Albers",
    BASEGEOGCRS["NAD83",
        DATUM["North American Datum 1983",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            

In [ ]:
# # Find all NTF files in subdirectories
# # ntf_files = glob.glob("**/*.ntf", recursive=True)

# # Ensure we have NTF files
# if not ntf_files:
#     raise ValueError("❌ No .ntf files found!")

# print(f"✅ Found {len(ntf_files)} NTF files.")

# # Initialize a list to store acquisition dates
# acquisition_dates = []
# # Dictionary to count occurrences of VNIR catid values
# vnir_counter = Counter()

# # Loop through each NTF file to extract the acquisition date
# for ntf_file in ntf_files:
#     dataset = gdal.Open(ntf_file)
#     if dataset:
#         metadata = dataset.GetMetadata()
#         date = metadata.get("NITF_STDIDC_ACQUISITION_DATE")
#         if date:
#             acquisition_dates.append(date)
#             # print(f"📅 {ntf_file}: {date}")  # Show extracted date
#         sensor = metadata.get("NITF_ISORCE")
#         dataset = None  # Close dataset

#         # Extract VNIR value
#         if "NITF_PIAIMC_SOURCE" in metadata:
#             source_metadata = metadata["NITF_PIAIMC_SOURCE"]
            
#             if "VNIR:" in source_metadata:
#                 for line in source_metadata.split("\n"):  # Handle multi-line metadata
#                     if line.startswith("VNIR:"):
#                         vnir_value = line.split(":", 1)[1].strip()  # Extract value after colon
#                         vnir_counter[vnir_value] += 1  # Count occurrences

# # Find the minimum date
# if acquisition_dates:
#     min_date = min(acquisition_dates)  # Dates are stored as strings, lexicographically sorted works for YYYYMMDD
#     max_date = max(acquisition_dates)  # Dates are stored as strings, lexicographically sorted works for YYYYMMDD
#     print(f"\n✅ Earliest acquisition date: {min_date}")
#     print(f"\n✅   Latest acquisition date: {max_date}")
# else:
#     print("❌ No acquisition dates found in metadata!")

# # If only one unique VNIR value exists, print it
# if len(vnir_counter) == 1:
#     catid = next(iter(vnir_counter))  # Get the single VNIR value
#     print(f"\nCatalog ID: {catid}")
# else:
#     print("Multiple catalog ids, exiting.")
#     sys.exit()  # Exit without printing anything

# timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# output_name = f"MS4_2m_{min_date}_{sensor}_{catid}_v{timestamp}"

# print(f"\nOutput Filename: {output_name}")


In [ ]:
# # Function to determine the number of bands in an NTF file
# def get_num_bands(ntf_file):
#     dataset = gdal.Open(ntf_file)
#     if dataset:
#         num_bands = dataset.RasterCount
#         dataset = None  # Close the dataset
#         return num_bands
#     else:
#         raise ValueError(f"❌ Could not open {ntf_file}")

# # Check the band count of the first NTF file (assumes all have the same structure)
# num_bands = get_num_bands(ntf_files[0])
# print(f"✅ Detected {num_bands} bands.")

# # Define which bands to process
# bands_to_process = list(range(1, 5)) if num_bands == 4 else [2, 3, 5, 7]

# # -t_srs EPSG:${epsg} -multi -wo NUM_THREADS=8 -co NUM_THREADS=8 --config GDAL_CACHEMAX 1536 
# # -wm 1536 -rpc -to RPC_DEM=${DEM_FILE} -srcnodata 0 -of cog -tr 2 2 -tap -r cubic 
# # -co "PREDICTOR=2" -co "BIGTIFF=IF_SAFER" ${inFilePath} ${outFile}


# # Construct the gdalwarp command
# warp_cmd = [
#     # conda_path, "run", "-n", conda_env,
#     # "gdalwarp",
#     "/home/mmacander/miniconda3/envs/cog/bin/gdalwarp",
#     "-t_srs", "EPSG:3338",  # Set target spatial reference
#     "-multi",
#     "-wo", "NUM_THREADS=8",
#     "-co", "NUM_THREADS=8",
#     "--config", "GDAL_CACHEMAX 1536",
#     "-wm", "1536",
#     "-rpc",  # Use Rational Polynomial Coefficients
#     "-to", f"RPC_DEM={dem_tif}",  # Use the specified DEM
#     "-srcnodata", "0",
#     "-of", "COG",  # Output format is COG
#     "-tr", "2", "2",
#     "-tap",
#     "-r", "cubic",  # Resampling method
#     "-co", "PREDICTOR=2",  # Apply LZW compression predictor
#     "-co", "BIGTIFF=IF_SAFER"  # Enable BigTIFF support
# ]

# # Add band selection options
# for band in bands_to_process:
#     warp_cmd.extend(["-srcband", str(band)])

# # Add all found input NTF files and output filename
# warp_cmd.extend(ntf_files)
# warp_cmd.extend([output_cog])

# # Print and execute the command
# print(f"🚀 Running: {' '.join(warp_cmd)}")
# subprocess.run(warp_cmd, shell=True, executable="/bin/bash", check=True)
# # subprocess.run(warp_cmd, check=True)


# print(f"✅ gdalwarp completed! Output COG saved as: {output_cog}")